In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
import joblib



In [2]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']

    df.duration = df.duration.apply(lambda row: row.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <=60)]

    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] = df[categorical].astype(str)
    
    return df

In [3]:
df_train = read_dataframe('data/green_tripdata_2022-01.parquet')
df_val = read_dataframe('data/green_tripdata_2022-02.parquet')

In [5]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# DictVectorizer - it vetorize the dictionary
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [6]:
target = 'duration'
y_train = df_train[target].to_numpy()
y_val = df_val[target].to_numpy()

In [7]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

8.193832557405061

In [11]:
lr = Lasso(alpha=0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

8.526684901296404

## Saving model

In [15]:
artifacts = {
    "model": lr,
    "dv": dv
}

joblib.dump(artifacts, "models/model.bin")


['models/model.bin']

## Loading model

In [63]:
artifacts = joblib.load("model.bin")

model = artifacts["model"]
dv = artifacts["dv"]


In [ ]:
sample = {
    "PULocationID": "130",
    "DOLocationID": "205",
    "trip_distance": 3.2
}

X = dv.transform([sample])
prediction = model.predict(X)

print(prediction)
